In diesem Notebook werden auf verschiedenen Modalitäten verschiedene Modelle trainiert, angewendet und ausgewertet. Jeweils: Random Forest, XGBoost, Logistic Regression, SVM, Neuronales Netz

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score
from sklearn.dummy import DummyClassifier

# Vorbereitung: Data Split in 3 Datasets!

Preprocessing der Kategorsischen Variablen..

In [ ]:
df = pd.read_csv("features_enriched_tab_img_text.csv")
print(df.shape)
print(df.columns)

df = df.rename(
    columns={
        "stars": "tab_star",
        "year": "tab_year",
        "month": "tab_month",
        "day": "tab_day",
        "likesCount": "tab_likesCount",
        "reviewerNumberOfReviews": "tab_reviewerNumberOfReviews",
        "isLocalGuide": "tab_isLocalGuide",
        "dayofweek": "tab_dayofweek",
        "elapsed_days": "tab_elapsed_days",
        "when_on_day_4hbin": "tab_when_on_day_4hbin",
    }
)

df["label"] = df["label"].replace({"real": 0, "fake": 1})
df["text_sentiment"] = df["text_sentiment"].replace(
    {"negative": 0, "neutral": 1, "positive": 2}
)

df_num = df.select_dtypes(include=["float64", "int64"])
df_cat = df.select_dtypes(include=["object"])

df.to_csv("features_enriched_tab_img_text_preproc.csv", index=False)

In [ ]:
df = pd.read_csv("features_enriched_tab_img_text_preproc.csv")

print(df.shape)
print(df.columns)
print(df.filter(regex='text_'))

Train-Validation-Test-Split.

In [45]:
df = pd.read_csv('features_enriched_tab_img_text_preproc.csv')
print(f"Orginal:\t{df.shape}")
print("---------------------------------------------------------------------------------------------")

train, test = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=1)
train_val, test = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=1)
train, val = train_test_split(train_val, test_size=0.2, stratify=train_val['label'], random_state=1)
print(f"Train:\t\t{train.shape}")
print(f"Validation:\t{val.shape}")
print(f"Test:\t\t{test.shape}")

train.to_csv('detection_train.csv', index=False)
val.to_csv('detection_val.csv', index=False)
test.to_csv('detection_test.csv', index=False)
train_val.to_csv('detection_train_val.csv', index=False)

Orginal:	(1408, 574)
---------------------------------------------------------------------------------------------
Train:		(900, 574)
Validation:	(226, 574)
Test:		(282, 574)


## Baseline mit Dummy-Klassifikator: Coinflip

In [ ]:
from sklearn.dummy import DummyClassifier

train_df = pd.read_csv('detection_train.csv')
val_df = pd.read_csv('detection_val.csv')
test_df = pd.read_csv('detection_test.csv')

used_features = train_df.filter(regex='^(feature)').columns

X_train, y_train = train_df[used_features], train_df['label']
X_val, y_val = val_df[used_features], val_df['label']
X_test, y_test = test_df[used_features], test_df['label']

dummy = DummyClassifier(strategy='uniform', random_state=1)
dummy.fit(X_train, y_train)

y_pred_val = dummy.predict(X_val)
y_pred_test = dummy.predict(X_test)

print(f"____________________________EVALUATION DUMMY CLASSIFICATOR BASELINE_________________________________________________")
accuracy_val = accuracy_score(y_val, y_pred_val)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
confusion = confusion_matrix(y_test, y_pred_test)
auc = roc_auc_score(y_test, y_pred_test)
print(f"Validation set accuracy: {accuracy_val}")
print(f"Test set accuracy: {accuracy_test}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")
print(f"AUC: {auc}")
print(f"Confusion matrix:\n{confusion}")


# TAB

XGBoost

In [1]:
from xgboost import XGBClassifier
import shap

train_df = pd.read_csv('detection_train.csv')
val_df = pd.read_csv('detection_val.csv')
test_df = pd.read_csv('detection_test.csv')

used_features = train_df.filter(regex='^(feature_)').columns

X_train, y_train = train_df[used_features], train_df['label']
X_val, y_val = val_df[used_features], val_df['label']
X_test, y_test = test_df[used_features], test_df['label']

# Perform normalization
X_train_norm = (X_train - X_train.mean()) / X_train.std()
X_val_norm = (X_val - X_train.mean()) / X_train.std()
X_test_norm = (X_test - X_train.mean()) / X_train.std()

# #Perform standardization
# scaler = StandardScaler()
# X_train_norm = scaler.fit_transform(X_train)
# X_val_norm = scaler.transform(X_val)
# X_test_norm = scaler.transform(X_test)'


xgb = XGBClassifier(random_state=3, n_estimators=1000)
xgb.fit(X_train_norm, y_train)
y_pred_val = xgb.predict(X_val_norm)
y_pred_test = xgb.predict(X_test_norm)

accuracy_val = accuracy_score(y_val, y_pred_val)
accuracy_test = accuracy_score(y_test, y_pred_test)
f1 = f1_score(y_val, y_pred_val)
precision = precision_score(y_val, y_pred_val)
recall = recall_score(y_val, y_pred_val)
confusion = confusion_matrix(y_val, y_pred_val)
auc = roc_auc_score(y_val, y_pred_val)
print(f"Validation set accuracy: {accuracy_val}")
print(f"Test set accuracy: {accuracy_test}")
print(f"F1 score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Confusion matrix:\n{confusion}")
print(f"AUC: {auc}")
print(f"________________________________________________________________________")

explainer = shap.Explainer(xgb, X_train_norm, feature_names=X_train.columns)
shap_values = explainer(X_train_norm)

shap.plots.beeswarm(shap_values, max_display=10,show=False) 
plt.tight_layout()
plt.savefig('shap_tab_xgb.svg', format='svg', dpi=1200)

c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


NameError: name 'pd' is not defined

# Tabular

# Image

# Multimodal